<h1><i> Final Capstone Project of Coursera </i></h1>

<h4> 1. Description problem </h4>

<p> In this final project we will help to an Bank to evaluate where can open in New York. </br>
Recently the bank is operating in Toronto. For the Executive Directory it's important know how the City of New York is clustering, looking for the principal businnes are exits in Manhattan.</p>

<p> Based on the New York Neighborhoods, we have to explore and find some similarys Neighbourhoods, compared with Toronto. </p>

<h3> 2. About the Data </h3>

<h4> 2.1 New York Dataset </h4>

<p> First, we will download the Neighborhoods dataset from <a href="https://geo.nyu.edu/catalog/nyu_2451_34572">New York: A City of Neighborhoods.</a></p>

In [17]:
import json
import pandas as pd

!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
    
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)   

neighborhoods_data = newyork_data['features']
df_n = pd.DataFrame()

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    df_n = df_n.append({'Borough': borough,
                        'Neighborhood': neighborhood_name,
                        'Latitude': neighborhood_lat,
                        'Longitude': neighborhood_lon,
                        'City': 'New York'
                       }, ignore_index=True)

df_n = df_n[['Borough','Neighborhood','Latitude','Longitude','City']]    
df_n.head()

,Borough,Neighborhood,Latitude,Longitude,City
0,Bronx,Wakefield,40.894705,-73.847201,New York
1,Bronx,Co-op City,40.874294,-73.829939,New York
2,Bronx,Eastchester,40.887556,-73.827806,New York
3,Bronx,Fieldston,40.895437,-73.905643,New York
4,Bronx,Riverdale,40.890834,-73.912585,New York


<p> Then we put the data on a Folium Map to visualize the New York Neighborhoods. </p>

In [81]:
import folium
from geopy.geocoders import Nominatim

manhattan_data = df_n[df_n['Borough'] == 'Manhattan'].reset_index(drop=True)
address = 'Manhattan, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudeNY = location.latitude
longitudeNY = location.longitude

map_manhattan = folium.Map(location=[latitudeNY, longitudeNY], zoom_start=11)

for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

<h4> 2.2 Toronto Dataset </h4>

<p> In second place, we need the neighbourhoods from Toronto, so for that we are going to download the postal code of each neighbourhood from Wikipedia through Web Scraping using BeautifulSoup libraries.</p>

In [3]:
import requests
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup

postal_codes = []

req = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(req.text, "html.parser")

postal_table = soup.find('table',{"class":"wikitable sortable"})

for row in postal_table.find_all('tr'):
    cols = row.find_all('td')
    if len(cols) == 3:
        postal_codes.append((cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip(),'Toronto'))

df_t = pd.DataFrame(postal_codes)
df_t.columns = ['Postcode', 'Borough','Neighbourhood','City']
df_t = df_t[df_t.Borough != 'Not assigned']
df_t = df_t.rename(columns={'Postcode': 'PostalCode'})
df_t.loc[df_t['Neighbourhood'] == "Not assigned", 'Neighbourhood'] = df_t['Borough']
df_t.head()

,PostalCode,Borough,Neighbourhood,City
2,M3A,North York,Parkwoods,Toronto
3,M4A,North York,Victoria Village,Toronto
4,M5A,Downtown Toronto,Harbourfront,Toronto
5,M5A,Downtown Toronto,Regent Park,Toronto
6,M6A,North York,Lawrence Heights,Toronto


<p> Then we JOIN the Toronto dataset with Geospatial data obtained from Cognitive Class.</p>

In [4]:
!wget -q -O 'canada_data.csv' https://cocl.us/Geospatial_data

df_coordinates = pd.read_csv("canada_data.csv") 
df_coordinates = df_coordinates.rename(columns={'Postal Code':'PostalCode'})
df_coordinates = pd.merge(df_t, df_coordinates, on='PostalCode', how='inner')
df_coordinates = df_coordinates[['Borough','Neighbourhood','Latitude','Longitude','City']]   
df_coordinates = df_coordinates.rename(columns={'Neighbourhood':'Neighborhood'})

toronto_data = df_coordinates[df_coordinates['Borough'].str.contains('Toronto')]
df_t  = toronto_data
df_t = df_t.reset_index(drop=True)
df_t.head()

,Borough,Neighborhood,Latitude,Longitude,City
0,Downtown Toronto,Harbourfront,43.654260,-79.360636,Toronto
1,Downtown Toronto,Regent Park,43.654260,-79.360636,Toronto
2,Downtown Toronto,Ryerson,43.657162,-79.378937,Toronto
3,Downtown Toronto,Garden District,43.657162,-79.378937,Toronto
4,Downtown Toronto,St. James Town,43.651494,-79.375418,Toronto


<p> And same as New York, we put the data on a Folium Map to visualize the Neighborhoods in Toronto </p>

In [5]:
import folium 
from geopy.geocoders import Nominatim 

address = 'Toronto, ON'
geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitudeTO = location.latitude
longitudeTO = location.longitude

map_canada = folium.Map(location=[latitudeTO, longitudeTO], zoom_start=11)

for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

<h4> 3.0 Brief summary </h4>

In [14]:
print('The data set from New York has {} rows.'.format(df_n.count().unique()))
print('The data set from Toronto has {} rows.'.format(df_t.count().unique()))

print('New York has {} unique Borough.'.format(len(df_n['Borough'].unique())))
print('Toronto has {} unique Borough.'.format(len(df_t['Borough'].unique())))

The data set from New York has [306] rows.
The data set from Toronto has [74] rows.
New York has 5 unique Borough.
Toronto has 4 unique Borough.


<h4> 2.3 Mix both DataFrame in a single one </h4>

In [15]:
neighborhoods = pd.concat([df_n, df_t])
neighborhoods = neighborhoods.reset_index(drop=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude,City
0,Bronx,Wakefield,40.894705,-73.847201,New York
1,Bronx,Co-op City,40.874294,-73.829939,New York
2,Bronx,Eastchester,40.887556,-73.827806,New York
3,Bronx,Fieldston,40.895437,-73.905643,New York
4,Bronx,Riverdale,40.890834,-73.912585,New York


<h3> 4. Methodology </h3>

<h4> 4.1 Connecting to API Foursquare </h4>

<p> We will use the Foursquare API to get the nearby venues and k-means clustering algorithm to analyze the Neighbourhood.</p>

In [24]:
#faby
#CLIENT_ID = 'EONVGZTB5GGOGRKJUOAI3EPK5NHLEJ4FM3KCAFJT1NVLTOVL'
#CLIENT_SECRET = 'GFI2LJISK24GRFXKLRWLM1BWQGKRVHKEX4HVZVKEPSV2C32R'

#yuan
CLIENT_ID = 'K2WBN4XXZVC0UWD3QX3EPUOM5KC3RNA1YCGG4FQ2GOH4RR3L'
CLIENT_SECRET = 'CLMC2ED2SFQ0JHDY5TCXBJ5O5ZKYDYQWL2DNYLAANPDOP3K5'

VERSION = '20190314'
LIMIT = 100
radius = 500

<p> Then define an function to download the nearby venues from the API Foursquare. </p>

In [25]:
def getNearbyVenues(names, latitudes, longitudes, city, radius):

    venues_list = []

    for name, lat, lng, city in zip(names, latitudes, longitudes, city):

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                                    CLIENT_ID, 
                                    CLIENT_SECRET, 
                                    VERSION, 
                                    lat,
                                    lng,
                                    radius, 
                                    LIMIT)

        results = requests.get(url).json()    
        venues = results['response']['groups'][0]['items']

        venues_list.append([(
            name, 
            lat, 
            lng,
            city,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venues])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                      'Neighborhood Latitude', 
                      'Neighborhood Longitude', 
                      'Neighborhood City', 
                      'Venue', 
                      'Venue Latitude', 
                      'Venue Longitude', 
                      'Venue Category']
    #error Groups tiene relación con la cuota excedida con la API de Foursquare
    return(nearby_venues)

<p> We called the function to obtain de nearby venues from the DataFrame Neighborhoods. </p>

<h4> 4.2 Download data from Foursquare </h4>

In [26]:
neighborhoods_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'],
                                   city=neighborhoods['City'],    
                                   radius=500
                                  )

<p> Below show the existing venues from each Neighborhood with a Radius of 500. </p>

In [30]:
neighborhoods_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Neighborhood City,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Adelaide,100,100,100,100,100,100,100
Allerton,27,27,27,27,27,27,27
Annadale,9,9,9,9,9,9,9
Arden Heights,5,5,5,5,5,5,5
Arlington,5,5,5,5,5,5,5


<p> Now look for how many uniques categories of venues. </p>

In [29]:
print('There are {} uniques categories.'.format(len(neighborhoods_venues['Venue Category'].unique())))

There are 448 uniques categories.


<h3> 5. Analyze Toronto and New York Neighborhoods </h3>

In [31]:
# one hot encoding
neighborhoods_onehot = pd.get_dummies(neighborhoods_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhoods_onehot['Neighborhood'] = neighborhoods_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neighborhoods_onehot.columns[-1]] + list(neighborhoods_onehot.columns[:-1])
neighborhoods_onehot = neighborhoods_onehot[fixed_columns]

neighborhoods_onehot.head()

,Yoga Studio,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Warehouse Store,Waste Facility,Watch Shop,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<p> This is our new dataframe with the neighborhoods. </p>

In [33]:
neighborhoods_onehot.shape

(13521, 448)

<p> Analyze the mean of the frequency of occurrence of each category grouped by Neighborhood. </p>

In [35]:
neighborhoods_grouped = neighborhoods_onehot.groupby('Neighborhood').mean().reset_index()
neighborhoods_grouped.head()

,Neighborhood,Yoga Studio,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,...,Warehouse Store,Waste Facility,Watch Shop,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adelaide,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01
1,Allerton,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
2,Annadale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
3,Arden Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
4,Arlington,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00


<h4> 5.1 Search for Top 5 venues in each neighborhood </h4>

In [37]:
num_top_venues = 5

for hood in neighborhoods_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = neighborhoods_grouped[neighborhoods_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
             venue  freq
0      Coffee Shop  0.06
1             Café  0.05
2              Bar  0.04
3  Thai Restaurant  0.04
4       Steakhouse  0.04


----Allerton----
          venue  freq
0   Supermarket  0.11
1        Bakery  0.07
2   Pizza Place  0.07
3           Spa  0.07
4  Liquor Store  0.07


----Annadale----
           venue  freq
0    Pizza Place  0.22
1  Train Station  0.11
2     Restaurant  0.11
3          Diner  0.11
4     Sports Bar  0.11


----Arden Heights----
           venue  freq
0       Pharmacy   0.2
1    Pizza Place   0.2
2  Deli / Bodega   0.2
3    Coffee Shop   0.2
4       Bus Stop   0.2


----Arlington----
                 venue  freq
0             Bus Stop   0.2
1        Boat or Ferry   0.2
2        Grocery Store   0.2
3          Coffee Shop   0.2
4  American Restaurant   0.2


----Arrochar----
                venue  freq
0            Bus Stop  0.15
1  Italian Restaurant  0.10
2       Deli / Bodega  0.10
3         Pizza Place  0.10
4      Cos

<h4> 5.2 Define a function to sort the venues in descending order </h4>

In [56]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<p> Now we are going to look for the top 10 venues for each neighborhood. </p>

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neighborhoods_grouped['Neighborhood']

for ind in np.arange(neighborhoods_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhoods_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Asian Restaurant,Gym,Bakery,American Restaurant,Burger Joint
1,Allerton,Supermarket,Bakery,Liquor Store,Spa,Pizza Place,Breakfast Spot,Playground,Martial Arts Dojo,Grocery Store,Chinese Restaurant
2,Annadale,Pizza Place,Sports Bar,Restaurant,American Restaurant,Train Station,Dance Studio,Diner,Park,English Restaurant,Ethiopian Restaurant
3,Arden Heights,Pharmacy,Pizza Place,Deli / Bodega,Bus Stop,Coffee Shop,Factory,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant
4,Arlington,Boat or Ferry,Bus Stop,Grocery Store,Coffee Shop,American Restaurant,Food,Flower Shop,Empanada Restaurant,English Restaurant,Ethiopian Restaurant


<h3> 6. Cluster Neighborhoods </h3>

<p> Then we run k-means to cluster into 10 clusters the neighborhood. </p>

In [59]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 10

neighborhoods_grouped_clustering = neighborhoods_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhoods_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([8, 4, 4, 7, 7, 9, 4, 8, 9, 4], dtype=int32)

<p> Now create an dataframe that includes the cluster with the top 10 venues for each neighborhood. </p>

In [61]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

<p> We check the new dataframe. </p>

In [63]:
neighborhoods_merged = neighborhoods
neighborhoods_merged.head()

,Borough,Neighborhood,Latitude,Longitude,City
0,Bronx,Wakefield,40.894705,-73.847201,New York
1,Bronx,Co-op City,40.874294,-73.829939,New York
2,Bronx,Eastchester,40.887556,-73.827806,New York
3,Bronx,Fieldston,40.895437,-73.905643,New York
4,Bronx,Riverdale,40.890834,-73.912585,New York


<p> Merge the new dataframe with the venues sorted ones.</p>

In [64]:
neighborhoods_merged = neighborhoods_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

<p> Then create two data frame to separate the neighborhoods from New York and Toronto. </p>

In [65]:
newyork_merged = neighborhoods_merged[neighborhoods_merged['City'].str.contains('New York') == True]
toronto_merged = neighborhoods_merged[neighborhoods_merged['City'].str.contains('Toronto') == True]

In [66]:
newyork_merged = newyork_merged.rename(columns={'Cluster Labels':'ClusterLabels'})
toronto_merged = toronto_merged.rename(columns={'Cluster Labels':'ClusterLabels'})
newyork_merged = newyork_merged[newyork_merged['ClusterLabels'].notnull()]
toronto_merged = toronto_merged[toronto_merged['ClusterLabels'].notnull()]

<p> Put the data cluster on a Folium Map to visualize. </p>

In [67]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

# create map
map_clusters_ny = folium.Map(location=[latitudeNY, longitudeNY], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newyork_merged['Latitude'], newyork_merged['Longitude'], newyork_merged['Neighborhood'], newyork_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters_ny)
       
map_clusters_ny

<p> Repeat the same with Toronto cluster. </p>

In [69]:
# create map of Toronto
map_clusters_to = folium.Map(location=[latitudeTO, longitudeTO], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters_to)
       
map_clusters_to

<h3> 7. Examine the cluster </h3>

In [71]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 0,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
183,Jamaica Estates,New York,0.0,Intersection,Lounge,Indian Restaurant,Farm,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service


In [72]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 1,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Fieldston,New York,1.0,River,Playground,Plaza,Women's Store,Factory,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant
27,Clason Point,New York,1.0,Park,Boat or Ferry,Grocery Store,South American Restaurant,Pool,Bus Stop,Scenic Lookout,Women's Store,Eye Doctor,Electronics Store
60,Brownsville,New York,1.0,Restaurant,Park,Check Cashing Service,Burger Joint,Chinese Restaurant,Donut Shop,Trail,Discount Store,Spanish Restaurant,Playground
81,Dyker Heights,New York,1.0,Playground,Plaza,Bagel Shop,Burger Joint,Hot Dog Joint,Golf Course,Dance Studio,Event Space,Eye Doctor,Exhibit
203,Todt Hill,New York,1.0,Trail,Park,Women's Store,Duty-free Shop,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service
294,Malba,New York,1.0,Rest Area,Tennis Court,Bus Line,Women's Store,Falafel Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
303,Bayswater,New York,1.0,Park,Playground,Tennis Court,Women's Store,Factory,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
337,Lawrence Park,Toronto,1.0,Swim School,Park,Bus Line,Women's Store,Falafel Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service
340,Forest Hill North,Toronto,1.0,Trail,Sushi Restaurant,Jewelry Store,Park,Women's Store,Falafel Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant
341,Forest Hill West,Toronto,1.0,Trail,Sushi Restaurant,Jewelry Store,Park,Women's Store,Falafel Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant


In [73]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 2,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
207,Port Ivory,New York,2.0,Bar,Women's Store,Fountain,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space
212,Oakwood,New York,2.0,Bar,Bus Stop,Women's Store,Fountain,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space


In [74]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 3,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
192,Somerville,New York,3.0,Park,Women's Store,Irish Pub,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space


In [75]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 4,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,New York,4.0,Pharmacy,Sandwich Place,Food,Caribbean Restaurant,Donut Shop,Laundromat,Dessert Shop,Food Truck,Ice Cream Shop,Cycle Studio
1,Co-op City,New York,4.0,Bus Station,Grocery Store,Baseball Field,Fast Food Restaurant,Basketball Court,Pizza Place,Restaurant,Pharmacy,Ice Cream Shop,Park
2,Eastchester,New York,4.0,Caribbean Restaurant,Bus Station,Diner,Bowling Alley,Chinese Restaurant,Metro Station,Donut Shop,Seafood Restaurant,Pizza Place,Platform
5,Kingsbridge,New York,4.0,Pizza Place,Sandwich Place,Bar,Mexican Restaurant,Discount Store,Supermarket,Latin American Restaurant,Pharmacy,Donut Shop,Bakery
8,Norwood,New York,4.0,Pizza Place,Park,Bank,Pharmacy,Mexican Restaurant,Sandwich Place,Coffee Shop,Mobile Phone Shop,Caribbean Restaurant,Bus Stop
10,Baychester,New York,4.0,Electronics Store,Playground,Bank,Food & Drink Shop,Gym / Fitness Center,Men's Store,Donut Shop,Pet Store,Mexican Restaurant,Sandwich Place
11,Pelham Parkway,New York,4.0,Chinese Restaurant,Frozen Yogurt Shop,Italian Restaurant,Pizza Place,Sandwich Place,Sushi Restaurant,Eye Doctor,Metro Station,Performing Arts Venue,Bakery
13,Bedford Park,New York,4.0,Diner,Mexican Restaurant,Deli / Bodega,Pizza Place,Chinese Restaurant,Supermarket,Fried Chicken Joint,Pharmacy,Spanish Restaurant,Sandwich Place
14,University Heights,New York,4.0,Pizza Place,Fried Chicken Joint,Bus Station,Grocery Store,Bank,Bakery,Chinese Restaurant,Donut Shop,Pharmacy,Sandwich Place
15,Morris Heights,New York,4.0,Food Truck,Spanish Restaurant,Latin American Restaurant,Pizza Place,Pharmacy,Bus Station,Food,Recreation Center,Supermarket,Bank


In [76]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 5,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
225,Westerleigh,New York,5.0,Convenience Store,Dance Studio,Arcade,Farm,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service


In [77]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 6,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
365,CN Tower,Toronto,6.0,Airport Lounge,Airport Terminal,Airport Service,Airport Food Court,Harbor / Marina,Sculpture Garden,Airport Gate,Boutique,Airport,Boat or Ferry
366,Bathurst Quay,Toronto,6.0,Airport Lounge,Airport Terminal,Airport Service,Airport Food Court,Harbor / Marina,Sculpture Garden,Airport Gate,Boutique,Airport,Boat or Ferry
367,Island airport,Toronto,6.0,Airport Lounge,Airport Terminal,Airport Service,Airport Food Court,Harbor / Marina,Sculpture Garden,Airport Gate,Boutique,Airport,Boat or Ferry
368,Harbourfront West,Toronto,6.0,Airport Lounge,Airport Terminal,Airport Service,Airport Food Court,Harbor / Marina,Sculpture Garden,Airport Gate,Boutique,Airport,Boat or Ferry
369,King and Spadina,Toronto,6.0,Airport Lounge,Airport Terminal,Airport Service,Airport Food Court,Harbor / Marina,Sculpture Garden,Airport Gate,Boutique,Airport,Boat or Ferry
370,Railway Lands,Toronto,6.0,Airport Lounge,Airport Terminal,Airport Service,Airport Food Court,Harbor / Marina,Sculpture Garden,Airport Gate,Boutique,Airport,Boat or Ferry
371,South Niagara,Toronto,6.0,Airport Lounge,Airport Terminal,Airport Service,Airport Food Court,Harbor / Marina,Sculpture Garden,Airport Gate,Boutique,Airport,Boat or Ferry


In [78]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 7,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
77,Manhattan Beach,New York,7.0,Bus Stop,Café,Ice Cream Shop,Playground,Pizza Place,Beach,Sandwich Place,Harbor / Marina,Food,English Restaurant
198,New Brighton,New York,7.0,Bus Stop,Park,Deli / Bodega,Convenience Store,Home Service,Bowling Alley,Playground,Discount Store,Empanada Restaurant,English Restaurant
205,Port Richmond,New York,7.0,Pizza Place,Bus Stop,Construction & Landscaping,Breakfast Spot,Bar,Donut Shop,Rental Car Location,Cycle Studio,Farmers Market,Ethiopian Restaurant
224,Park Hill,New York,7.0,Bus Stop,Athletics & Sports,Gym / Fitness Center,Park,Hotel,Coffee Shop,Women's Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
226,Graniteville,New York,7.0,Boat or Ferry,Wine Bar,Grocery Store,Moving Target,Bus Stop,Farm,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service
227,Arlington,New York,7.0,Boat or Ferry,Bus Stop,Grocery Store,Coffee Shop,American Restaurant,Food,Flower Shop,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
238,Butler Manor,New York,7.0,Pool,Baseball Field,Bus Stop,Convenience Store,Food & Drink Shop,Food Court,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
241,Arden Heights,New York,7.0,Pharmacy,Pizza Place,Deli / Bodega,Bus Stop,Coffee Shop,Factory,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant
242,Greenridge,New York,7.0,Bagel Shop,Pizza Place,Bus Stop,Diner,Women's Store,Falafel Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service
245,Bloomfield,New York,7.0,Recreation Center,Park,Theme Park,Discount Store,Bus Stop,Factory,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant


In [79]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 8,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Riverdale,New York,8.0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Park,Caribbean Restaurant,Cosmetics Shop,Indian Restaurant,Restaurant
6,Marble Hill,New York,8.0,Coffee Shop,Discount Store,Yoga Studio,Seafood Restaurant,Tennis Stadium,Big Box Store,Supplement Shop,Spa,Shoe Store,Donut Shop
9,Williamsbridge,New York,8.0,Caribbean Restaurant,Supermarket,Nightclub,Bar,Soup Place,Falafel Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
12,City Island,New York,8.0,Harbor / Marina,Thrift / Vintage Store,Seafood Restaurant,Deli / Bodega,Park,Music Venue,Smoke Shop,Diner,Pizza Place,Bank
24,Hunts Point,New York,8.0,Food,Waste Facility,Café,BBQ Joint,Farmers Market,Spanish Restaurant,Juice Bar,Restaurant,Pizza Place,Seafood Restaurant
28,Throgs Neck,New York,8.0,Juice Bar,Sports Bar,Asian Restaurant,American Restaurant,Deli / Bodega,Pizza Place,Liquor Store,Coffee Shop,Italian Restaurant,Chinese Restaurant
46,Bay Ridge,New York,8.0,Italian Restaurant,Spa,Pizza Place,Bagel Shop,Bar,Greek Restaurant,American Restaurant,Hookah Bar,Pharmacy,Sushi Restaurant
49,Greenpoint,New York,8.0,Bar,Pizza Place,Café,Cocktail Bar,Coffee Shop,Record Shop,Mexican Restaurant,Boutique,French Restaurant,Bakery
51,Brighton Beach,New York,8.0,Beach,Eastern European Restaurant,Restaurant,Gourmet Shop,Mobile Phone Shop,Pharmacy,Sushi Restaurant,Russian Restaurant,Fast Food Restaurant,Bank
52,Sheepshead Bay,New York,8.0,Dessert Shop,Turkish Restaurant,Sandwich Place,Boat or Ferry,American Restaurant,Grocery Store,Miscellaneous Shop,Café,Harbor / Marina,Diner


In [80]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 9,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Woodlawn,New York,9.0,Deli / Bodega,Playground,Pub,Pizza Place,Plaza,Bar,Supermarket,Bus Stop,Cosmetics Shop,Rental Car Location
23,Longwood,New York,9.0,Fast Food Restaurant,Community Center,Latin American Restaurant,Grocery Store,Donut Shop,Diner,Sandwich Place,Deli / Bodega,Women's Store,Exhibit
32,Van Nest,New York,9.0,Deli / Bodega,Pizza Place,Food Truck,Donut Shop,Spa,Playground,Spanish Restaurant,Hookah Bar,Coffee Shop,BBQ Joint
39,Edgewater Park,New York,9.0,Italian Restaurant,Deli / Bodega,Chinese Restaurant,Pizza Place,Coffee Shop,Farmers Market,Bar,Park,Ice Cream Shop,Pub
72,East New York,New York,9.0,Deli / Bodega,Plaza,Caribbean Restaurant,Chinese Restaurant,Event Service,Pizza Place,Latin American Restaurant,Spanish Restaurant,Gym,Fast Food Restaurant
83,Marine Park,New York,9.0,Gym,Park,Athletics & Sports,Ice Cream Shop,Baseball Field,Soccer Field,Basketball Court,Deli / Bodega,Pizza Place,Women's Store
89,Ocean Hill,New York,9.0,Deli / Bodega,Southern / Soul Food Restaurant,Playground,Convenience Store,Grocery Store,Food,Metro Station,Chinese Restaurant,Pharmacy,Donut Shop
148,South Ozone Park,New York,9.0,Deli / Bodega,Bar,Park,Food Truck,Fast Food Restaurant,Donut Shop,Hotel,Sandwich Place,Gym / Fitness Center,Event Space
150,Whitestone,New York,9.0,Train,Moving Target,Bubble Tea Shop,Deli / Bodega,Women's Store,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
159,Briarwood,New York,9.0,Deli / Bodega,Playground,Food Truck,Arts & Crafts Store,Plaza,Coffee Shop,Pet Store,Bus Station,Fast Food Restaurant,Mexican Restaurant
